<a href="https://colab.research.google.com/github/JHyunjun/TF2.0_AE_MLP/blob/main/MLPAE_tf2_0_220104.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Created by Hyunjun, JANG
# limitsinx.tistory.com

import numpy as np
import pandas as pd
import tensorflow as tf
import pandas_datareader.data as pdr
import matplotlib.pyplot as plt
import datetime
tf.random.set_seed(1)

#Data Load
df_train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data/AE/Train_1.txt', sep= "\t")
df_test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data/AE/Test_O.txt',sep= "\t")
df_test2 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data/AE/Test_X.txt',sep= "\t")
train_data = df_train.iloc[:,:]
test_data = df_test.iloc[:,:]
test_data2 = df_test2.iloc[:,:]
print(train_data)
print(test_data)
print(test_data2)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#Normalization
def MinMaxScaler(data):
  denom = np.max(data,0) - np.min(data,0)
  nume = data-np.min(data,0)
  return nume/denom

def back_MinMax(data,max,min):
  diff = max-min
  back = data * diff + min
  return back

In [ ]:
#Hyper parameters
seqLength = 0
dataDim = df_test.shape[1]
outputDim = 8
hiddenDim = 30
lr = 0.002
iterations = 10
batch_size = 16

trainSize = int(len(train_data))
trainSet = train_data[0:trainSize]
testSize = int(len(test_data))
testSet = test_data[0:testSize]
testSize2 = int(len(test_data2))
testSet2 = test_data2[0:testSize2]


testSet_for_result = testSet

trainSet_max = np.max(trainSet)
trainSet_min = np.min(trainSet)
testSet_max = np.max(testSet)
testSet_min = np.min(testSet)
trainSet_max = (trainSet_max[dataDim-1])
trainSet_min = (trainSet_min[dataDim-1])
testSet_max = (testSet_max[dataDim-1])
testSet_min = (testSet_min[dataDim-1])

trainSet = MinMaxScaler(trainSet)
testSet = MinMaxScaler(testSet)
testSet2 = MinMaxScaler(testSet2)

print(trainSet.shape) # (131874 ,8)
print(testSet.shape) # (20986, 8)
print(testSet2.shape) # (99669, 8)

In [ ]:
# Data Preprocessing
input_dim = trainSet.shape[1]
print(input_dim)
trainX = trainSet.iloc[:,0:input_dim] 
trainY = trainSet.iloc[:,0:input_dim]
testX = testSet.iloc[:,0:input_dim] 
testY = testSet.iloc[:,0:input_dim]
test2X = testSet2.iloc[:,0:input_dim]
test2Y = testSet2.iloc[:,0:input_dim]


In [ ]:
# Multi Layer Perceptron(MLP) Modeling
# Activation function "tanh" : Glorot normal initializer
# Activation function "reLU" : He Initiailizer
from tensorflow import keras
from tensorflow.keras import layers

initializer = tf.keras.initializers.GlorotNormal()
input_dim = input_dim
model = keras.Sequential()
model.add(layers.Dense(units = hiddenDim, activation='linear',input_dim = input_dim, kernel_initializer = initializer))
model.add(layers.Dense(units = hiddenDim, activation = 'linear', kernel_initializer = initializer))
model.add(layers.Dense(units = hiddenDim, activation = 'linear', kernel_initializer = initializer))
model.add(layers.Dense(units = hiddenDim, activation = 'linear', kernel_initializer = initializer))
model.add(layers.Dense(units = hiddenDim, activation = 'linear', kernel_initializer = initializer))
model.add(layers.Dense(units = hiddenDim, activation = 'linear', kernel_initializer = initializer))
model.add(layers.Dense(input_dim, activation='linear'))
adam = keras.optimizers.Adam(lr = lr)
model.compile(optimizer=adam, loss='mse')
model.summary()


In [ ]:
# Training
hist = model.fit(trainX, trainX, epochs = iterations * 2, batch_size = batch_size)

In [ ]:
# Evaluate Testset
res = model.evaluate(testX, testX, batch_size = batch_size)
print(res)
#res_batch_size1 = model.evaluate(testX, testY, batch_size = 1) #Very Slow


In [ ]:
# Estimating
xhat = testX
xhat2 = test2X
yhat = model.predict(xhat)
yhat2 = model.predict(xhat2)
testY = testY.to_numpy()
print("test Y : ", testY, testY.shape,type(testY))
print("yhat : ",yhat, yhat.shape,type(yhat))

print("RMSE : {}".format(np.sqrt(np.average((yhat - testY)**2))))
print(xhat.shape)

In [ ]:
estimated = back_MinMax(yhat,testSet_max,testSet_min) 
estimated= estimated[:,-1]
print("Estimated : ",estimated)
real = testSet_for_result.iloc[:,-1] # estimated + seqLength
print("REAL : ",real)
plt.figure()
plt.plot(estimated[:len(estimated)],label = "Estimated by MLP")
plt.plot(real[:len(real)], label = "Real")
plt.legend(prop = {'size':10})

In [ ]:
print("yhat : ",yhat.shape, yhat)
print("testY : ",testY.shape, testY)
print("test2Y : ",test2Y.shape, test2Y)

Loss = np.abs(np.abs(yhat) - np.abs(testY))
print("////////////////\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\/")
print("Loss : ",Loss.shape, Loss)
print("XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX")
Rank_Loss = Loss.sum(axis=1)
print("Rank Loss1 : ",Rank_Loss.shape, Rank_Loss)
Loss2 = np.abs(np.abs(yhat2)-np.abs(test2Y))
Rank_Loss2 = Loss2.sum(axis=1)

plt.figure()
plt.plot(Rank_Loss2[:len(Rank_Loss2)],label = "X_Abnormal")
plt.legend(prop = {'size':10})

plt.figure()
plt.plot(Rank_Loss[:len(Rank_Loss)],label = "O_Normal")
plt.legend(prop = {'size':10})